In [1]:
# imports
import pandas as pd
from thunderpack import ThunderReader
from tqdm import tqdm
import re
import os

This file should be run once.  It creates the icd partition and discharge csv's in order to free up ram.

# Admission Notes

In [2]:
year = 2010
output_dir = '/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cohortExtractionPipeline/BIDMC/CSVs'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

for i in tqdm(range(0, 15)):
    df = pd.read_csv(f'/home/jsearle/bigDrive/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_{year + i}_metadata.csv')
    df = df[df['NoteTypeFull'].str.contains('Initial Note', case=False, na=False)]
    df.to_csv(os.path.join(output_dir, f'admission_notes_{year + i}.csv'), index=False)


100%|██████████| 15/15 [00:55<00:00,  3.71s/it]


In [3]:
output_dir = '/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cohortExtractionPipeline/BIDMC/CSVs'

# Concatenate all filtered CSV files
all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.startswith('admission_notes_')]
admission_notes = pd.concat((pd.read_csv(f) for f in all_files), axis=0, ignore_index=True)

# Show number of total discharge summaries
print(len(admission_notes))

# Show example
admission_notes.head()

1981750


,BDSPPatientID,NoteTypeFull,Service,CreateDate,DeidentifiedName
0,150640302,Initial note,Orthopaedics,20180314,Notes_1130498758_1150758357_20180314.txt
1,150640136,Initial note,Rehabilitative Services,20180408,Notes_1130498996_10168670318_20180408.txt
2,150003905,Initial note,Vascular Surgery,20180421,Notes_1129862409_10172744188_20180421.txt
3,150643640,Initial note,Rehabilitative Services,20180524,Notes_1130502705_1153765704_20180524.txt
4,150643640,Initial note,Rehabilitative Services,20180524,Notes_1130502705_1153765705_20180524.txt


In [4]:
#filter by unique patient ID
admission_notes = set(admission_notes['BDSPPatientID'])
print(len(admission_notes))

329749


# ICD Partitions

In [5]:
reader = ThunderReader('/home/jsearle/bigDrive/Dropbox/zz_EHR_Thunderpacks/BIDMC/thunderpack_icd_9_10_nax_1m_BIDMC')
key_length = len(list(reader.keys()))
print(key_length)

36


In [6]:
output_dir = '/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cohortExtractionPipeline/BIDMC/CSVs'
os.makedirs(output_dir, exist_ok=True)

for i in tqdm(range(1, key_length + 1)):
    df = reader[f'ICD_partition_{i}']
    df = df[df['BDSPPatientID'].isin(admission_notes)]
    df.to_csv(os.path.join(output_dir, f'filtered_ICD_partition_{i}.csv'), index=False)

 19%|█▉        | 7/36 [00:52<03:38,  7.55s/it]


KeyboardInterrupt: 